# Práctica 1 Minería de Datos
# Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

Realizada por el **grupo S** formado por **Cristian Stanimirov Petrov** y **Nikola Svetlozarov Dyulgerov**



## Preparando entorno

Lo primero de todo importamos las librerías comunes con las que trabajaremos a lo largo de la práctica. Después importaremos alguna más especifica para apartados concretos, pero de momento con estas es suficiente.

**Fichero con nuestras funciones personalizadas**

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier

import md_grupoS_practica1_utils as utils

Establecemos una semilla para que se puedan reproducir los experimentos más adelante:

In [ ]:
seed = 562

## Pima Indians Diabetes

Esta base de datos ha sido construida con el objetivo de analizar una serie de **variables** para ver como están relacionadas con la aparición de diabetes. De esta manera lo que se pretende es poder predecir si un paciente tiene o no tiene dicha enfermedad en base a los factores descritos en el conjunto de datos.

Empezamos cargando los datos `pima_diabetes` para poder trabajar con ellos:

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"
# index = "Id"
target = "Outcome"
pima_diabetes = utils.load_data(filepath,target, "")
pima_diabetes.shape

Veamos una muestra aleatoria del conjunto de datos para ver que todo está en orden. Al ser aleatoria conseguimos más significancia estadística y evitamos las **muestras sesgadas** que juegan una mala pasada si queremos hacer algún tipo de verificación.

In [ ]:
pima_diabetes.sample(5, random_state=seed)

Bien ahora que sabemos nuestras **variables**, es una buena práctica separarlas en **predictoras** y **objetivo** para que su tratamiento sea más fácil y cómodo. Por convención, **X** son las predictoras e **y** las objetivo.

In [ ]:
(X,y) = utils.divide_dataset(pima_diabetes, target = "Outcome")

Comprobamos si se ha realizado correctamente la función antes de seguir.

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Antes de empezar con el análisis exploratorio separaremos nuestro conjunto de datos en conjunto de **entrenamiento** y  de **validación** para evitar  un **sobre-ajuste** del modelo a los datos. A esto se le llama *holdout* y para realzarlo usaremos un método del paquete de `sci-kit learn`.


In [ ]:
train_size = 0.7  #70% entrenamiento y 30% test
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Comprobamos como siempre que todo se ha realizado correctamente.

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
X_test.sample(5, random_state=seed)

Lo mismo con las variables objetivo de entrenamiento y prueba:

In [ ]:
y_train.sample(5, random_state=seed)

In [ ]:
y_test.sample(5, random_state=seed)

Para poder realizar el análisis exploratorio de una manera más simple, volveremos a juntar las variables predictoras con la objetivo para ambos conjuntos: entrenamiento y test

In [ ]:
data_train = utils.join_dataset(X_train, y_train)
data_test = utils.join_dataset(X_test, y_test)

Comprobamos ambos conjuntos

In [ ]:
data_train.sample(5, random_state=seed)

In [ ]:
data_test.sample(5, random_state=seed)

Ambas tablas son particiones de la original, veamos si la separación se ha ejecutado correctamente.

In [ ]:
data_train.shape

In [ ]:
data_test.shape

Todo correcto, ya que la suma de ambas **instancias** es igual al número total de instancias mostradas al cargar los datos por primera vez. Una vez listos los preliminares podemos proceder con el análisis exploratorio de la base de datos.

### Análisis exploratorio

Este apartado lo dividiremos en dos subpartes para poder entender con qué tipos de datos estamos tratando. Para ello analizaremos sus variables y la correlación entre ellas.

#### Descripción del conjunto de datos

Veamos el tipo de variables que tenemos que manejar. Esto nos dará pistas posteriormente sobre qué métodos podemos usar para su debido procesamiento.

Antes hemos revisado que hay un total de `768` casos y unas `9` variables, de las cuales `8` son predictores y `1` de clase. Pasemos a comprobar el tipo de variables de las que se trata.

In [ ]:
data_train.info(memory_usage=False)

Podemos observar que todas son numéricas, la mayoría de tipo entero y algunas de coma flotante. Esto nos da pistas de que posiblemente después tendresea necesario discretizarlas para poder obtener información relevante.

Por otro lado, las clases en las que se divide la variable objetivo son las siguientes:

In [ ]:
y_train.cat.categories


Entenderemos que `0` indica que **no** hay diabetes y un `1` a que **si**. 

#### Visualización de las variables

Como el análisis numérico es complicado, la manera más simple de sacar conclusiones rápidas acerca de los datos es mediante representaciones gráficas de estos. 

Al tener relativamente pocas variables, usaremos métodos **univariados** para visualizarlos.

Comenzamos con una descripción más detallada de cada variable:

In [ ]:
data_train.describe()

Analizando los datos extraidos de cada variable hay algo importante de lo que nos debemos percatar. Muchas variables tienen como valor mínimo el cero, algo que en la realidad es imposible dado el significado intrinseco de dichas variables médicas. Se trata de las siguientes caracterísitcas: **Glucose**, **BloodPressure**, **SkinThickness**, **Insulin** y **BMI**. Esto puede deberse a valores perdidos que luego se deberán tratar. Por otro lado, tenemos constancia de que la desviación media de **Insulin** es demasiado alta, lo que nos advierte de que hay mucho ruido en sus valores.

Veamos ahora como se traducen estos números en histogramas y diagramas de barras. Para ellos nos valemos de métodos que hemos construido con la librería `plotly` en nuestro archivo de utilidades.

In [ ]:
utils.plot_conditional_histogram(data_train, "Outcome")

Viendo los histogramas condicionados por la vairable de clase llegamos a las siguientes conclusiones:

*   **Glucose**, **BloodPressure** y **BMI** son variables que parece que intentan seguir una distribución normal aunque los `outliers` provocan una visualización distorsionada de esta.
*   Confirmamos la existencia de valores perdidos en algunas variables porque tienen puntos nulos que carecen de sentido: **SkinThickness**, **Insulin**, **BMI**, **Glucose** y **BloodPressure**.
*   La oblicuidad positiva o la derecha de las variables **DiabetesPedigreeFunction**, **Insulin** y **Pregnancies**. 
*   En el último histograma, el de la variable de clase, se ve claramente de que estamos lidiando con un problema desbalanceado. Esto nos hará recurrir a métodos de evaluación de clasificadores específicos para estas situaciones.
*   Se ha observado que en todas o casi todas las variables las distribuciones de cada categoría se superponen. Esto supone que la técnica de discretizado tendrá un efecto casi nulo, ya que carece de poder discriminatorio.



In [ ]:
utils.missing_values(data_train,["Glucose","BloodPressure", "SkinThickness","Insulin","BMI"])

La gráfica nos muestra el porcentaje de valores perdidos en las variables de las que hemos sospechado desde un principio. Tal y como se dijo en clase, en el prepocesamiento habrá que eliminar a **SkinThickness** e **Insulin**, ya que poseen más de un 20% de valores perdidos. Por tanto estas variables no las consideraremos a la hora de aprender nuestros modelos.

Hasta ahora hemos analizado las variables una por una, el siguiente paso es averiguar si hay algún tipo de relación entre ellas. Para ello nos valdremos de métodos **multivariados** para determinar cuales son las más relevantes en la predicción de diabetes al contraponerlas unas con otras. En nuestro caso, al no ser muy expertos creemos que un `pairplot` con tantas variables no nos dejaría nada claro, por eso recurrimos a su versión simplificada: el mapa de calor  o `heatmap`.

In [ ]:
utils.plot_heatmap(data_train.corr(), data_train.columns[:-2])

El gráfico nos ayuda a ver la correlación entre distintas variables, eso si, siempre es lineal con lo cual no nos daría información si hay algún otro tipo de dependencia entre ellas. A grosso modo se observa que las que mayor relación tienen son el par **Insulin* y **SkinThickness**, junto con el par **BMI** y **SkinThickness**. Por el otro lado, las demás correlaciones son muy bajas o casi nulas con lo cual da a entender que a simple vista es difícil percatarnos de cuales tienen un mayor poder predictivo.

### Preprocesamiento de datos

Pasamos a la fase mas importante del proceso. Para ello haremos uso del **pipeline** que se encargará de aplicar las transformaciones que creemos convenientes al conjunto de datos para después poder aprender un modelo concreto a partir él.

Lo primero de todo será eliminar las variables **SkinThickness** e **Insulin** tal y como especificamos en la parte del análisis exploratorio. Para ello recurrimos a uno de los transformadores del paquete de `sci-kit`.

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

column_transformer = make_column_transformer(("drop",["SkinThickness","Insulin"]),(SimpleImputer(missing_values = 0, strategy="mean"), ["Glucose","BloodPressure","BMI"]), remainder="passthrough")

Como dijimos anteriormente, en este conjunto de datos carece de sentido discretizar por lo que nos ahorramos dicha acción. Sin embargo, sigue habiendo un porcentaje de valores perdidos de algunas variables. En este caso lo que haremos es imputarlos (rellenar huecos). Tan solo lo aplicamos a las variables correspondientes, ya que si lo ejecutamos en todo el conjunto afectaría a otra variables en las que los valores nulos no son valores perdidos, digase el ejemplo de **Pregnancies**.

Con esto tendríamos los **datos crudos** preprocesados, en caso de necesitar otro tipo de preprocesamiento posteriormente tan solo habría que añadirlo al pipeline. 

### Zero-R

Tal y como se especifico usaremos este algoritmo "tonto" como punto de partida (baseline) para poder medir la efectividad de nuestro clasificador

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

### Árbol de decisión

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

Juntamos todo los pasos del preprocesamiento y el árbol de decisión en el pipeline

In [ ]:
pipeline = make_pipeline(column_transformer, tree_model)

### Evaluación

In [ ]:
utils.evaluate(zero_r_model, X_train, X_test, y_train, y_test)

Para ser un clasificador "tonto" ha conseguido un porcentaje de aciertos importante, aun así no es suficiente, pero nos valdrá como punto de partida. Por otro lado, hemos hecho uso del análisis ROC que nos indica claramente de que se trata de un mal clasificador al ser el mismo ejemplo visto en clase.

In [ ]:
utils.evaluate(pipeline, X_train, X_test, y_train, y_test)

Hay un importante mejora en la precisión del clasificador al crear el modelo a partir de datos ya procesados. Lo cual nos muestra la importancia de esta fase en el proceso que hemos llevado a cabo. Por otro lado, si nos fijamos en la matriz de confusión vemos que las tasas de falsos positivos y negativo es bastante alta por lo que podemos seguir mejorando intentando rebajar dicha cifras. La curva del análisis ROC ha mejorado notablemente también al haber disminuido los falsos positivos y aunmentado los verdaderos positivos, aun así sigue quedandose a medio camino de ser un muy buen clasificador. Como para esta práctica no veremos más tipos de preprocesamiento, no podremos ver cómo podría merjorar todavía mas.

## Breast Cancer Wisconsin

Esta base de datos está ideada en torno a variables médicas que se suelen utilizar para la detección de cáncer de mama, así que el fin con el que se construyó no es más que otro que intentar predecir su aparición de forma más precisa a través de técnias de machine learning.

### 1. Preliminares

Volvemos a importar las librerías que nos harán falta para "reiniciar" el entorno después de la sección anterior.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

# Aplicación local
import md_grupoS_practica1_utils as utils

Fijamos una semilla para poder reproducir el experimento al igual que antes

In [ ]:
seed = 1

### 2. Acceso y almacenamiento de datos

Cargamos nuestro conjunto de datos, en este caso el de `wisconsin` especificando que variable corresponde al identificador de las instancias del conjunto de datos y cual corresponde a la variable clase.

In [ ]:
# filepath = "../input/wisconsin.csv"
index = "id"
target = "diagnosis"

data = utils.load_data("../input/breast-cancer-wisconsin-data/data.csv", target, index)

Podríamos usar la función `head` para mostrar las primeras **n instancias** del conjunto de datos, pero sería una **muestra sesgada**. Para impedir que eso ocurra obtendremos una muestra aleatoria con la función `sample`

In [ ]:
data.sample(5, random_state=seed)

Como podemos observar en la muestra, tenemos una columna sin nombre (`Unnamed: 32`) que no tiene datos para niguna de las instancias, por lo tanto la eliminaremos puesto que no nos aporta nada.

In [ ]:
del data["Unnamed: 32"]
data.sample(5, random_state=seed)

Vamos a separar nuestro conjunto de datos en dos subconjuntos porque es util tener por un lado las variables predictoras y por otro lado la variable objetivo.

In [ ]:
(X,y) = utils.divide_dataset(data, target = target)

Y comprobamos que se haya realizado completamente. Primero, las variables predictoras:

In [ ]:
X.sample(5, random_state=seed)

Y por otro la variable clase

In [ ]:
y.sample(5, random_state=seed)

Puesto que no tenemos un conjunto de datos de prueba y para no provocar un sobre-ajuste al usar el mismo conjunto de datos para entrenar y realizar la prueba, lo separaremos como mínimo en dos:
* Una muestra de entrenamiento (70%)
* Una muestra de prueba (30%)

In [ ]:
train_size = 0.7
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Para facilitar el análisis exploratorio de datos, volvemos a juntar las variables predictoras con la variable clase. Comenzamos con el conjunto de datos de entrenamiento y luego procedemos igual para el conjunto de datos de prueba:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)
data_test = utils.join_dataset(X_test, y_test)

Comprobamos que se han juntado correctamente. Primero el conjunto de datos de entrenamiento:

In [ ]:
data_train.sample(5, random_state=seed)

Y después el conjunto de datos de prueba:

In [ ]:
data_test.sample(5, random_state=seed)

Y comprobamos que el conjunto de datos esta correctamente separado. Para ello usamos la función `shape` que nos muestra (número de instancias, número de variables)

In [ ]:
data.shape

In [ ]:
data_train.shape

In [ ]:
data_test.shape

Como podemos ver las 569 instancias del conjunto de datos original se han separado en:
* 398 para el conjunto de datos de entrenamiento
* 171 para el conjunto de datos de prueba

### 3. Análisis exploratorio de datos

Antes de realizar el preprocesamiento, observaremos las distintas variables que conforman la base de datos y las posibles relaciones entre ellas, mediante el apoyo de métodos gráficos y estadísticos.

#### Descripción del conjunto de datos

Como habíamos visto en el apartado anterior el conjunto de datos de entrenamiento esta formado por 398 instacias y 31 variables (30 variables predictoras y 1 variable clase).

In [ ]:
data_train.shape

Con la función `info` nos muestra las variables y de que tipo son:

In [ ]:
data_train.info(memory_usage=False)

Viendo esta información podemos afirmar que las 30 variables predictoras son numéricas (`float64`), y que la variable clase (`diagnosis`) es categórica (`category`). Otra de las cosas que nos percatamos es que en realidad se trata de 10 variables solamente, pero que se han convertido en 30 dado que son las diversas medidas tomadas de una misma: la media, el error estándar y por lo que hemos entendido en la descripción del conjunto de datos `worst` es la media de los tres valores más altos de cada variable.

Los estados de la la variable clase son:

In [ ]:
y_train.cat.categories

Hay dos clases y damos por entendido que **B** es benigno y **M** maligno.

#### Visualización estadística de los datos

Con la función `describe(include="number")` del paquete `Pandas` podemos analizar variables numéricas.

In [ ]:
data_train.describe(include="number")

Por lo general los datos de cada variable no son muy dispares, siendo las desviaciones estándar bajas salvo en el caso de las **areas** que suele ser bastante alta. Algo que entendemos dado los valores mínimos y máximos de estas. 

Con la función `describe(include="category")` podemos analizar variables numéricas.

In [ ]:
data_train.describe(include="category")

Con esto vemos que de los 398 casos, 250 son del tipo B y 148 del tipo M, lo que ya nos advierte de que se trata de un problema **desbalanceado**. Lo cual requerirá métodos de evaluación específicos que comentaremos en la última sección. 

#### Visualización gráfica de los datos

Pasemos ahora a la visualizaciones gráficas que son más sencillas de entender que solo mirando números. Para ello empezamos con los histogramas condicionados de cada variable.

In [ ]:
utils.plot_conditional_histogram(data_train, "diagnosis")

Viendo las histogramas de todas las variables predictoras llegamos a las siguientes conclusiones:


*   Lo primero de todo es que discretizar la mayoría de variables será en vano, ya que las clases se superponen en todo momento. Algunas como el **radio** y sus derivadas parece que serían una buena opción para discretizar porque es donde más separación se ve, pero aun así no es del todo clara. Lo mismo ocurre con **concave points**.
*   Se observan variables con cierta tendencia a una distribición normal, como **Smoothness_mean** y **Symmetry_mean**.
*   La oblicuidad a la derecha de todas las variables del error estándar.
*   La aparición de **outliers** en las variables del **radio** y las derivadas, se ve como se "arrastra" dicho ruido. También los valores anómalos están presentes en **Compactness**, **Smoothness** y **Fractal Dimension**

Por último, en general es notorio el desbalance del problema comentado anteriormente. Siempre la distribución de los casos benignos es superior al de malignos.

Como hemos podido ver, podemos diferenciar entre tres tipos de variables `mean`, `se` y `worst`, por lo tanto diseccionaremos nuestra base de datos para mostrar los mapas de calor ya que con treinta variables es dificil de visualizar si hay alguna correlación entre ellas.

In [ ]:
utils.plot_heatmap(X_train.corr(), X_train.iloc[:,:10].columns)

Viendo este mapa de correlación podemos confirmamos que `perimeter_mean` y `area_mean` dependen linealmente de `radius_mean`.

Lo mismo pasa con `concavity_mean`, `compactness_mean` y `concave points_mean` siento esta última la que tiene más correlación con otras variables.

In [ ]:
utils.plot_heatmap(X_train.corr(), X_train.iloc[:,10:20].columns)

Vuelva a ocurrir lo mismo que antes, pero esta vez con los valores del error estándar. Algo que por intuición podíamos deducir, ya que el error estándar proviene de las medias de las variables.

In [ ]:
utils.plot_heatmap(X_train.corr(), X_train.iloc[:,20:30].columns)

Una vez más la similitud con los mapas de calor anteriores es evidente a simple vista.

#### Preprocesamiento

Pasamos a la fase mas importante del proceso. Para ello haremos uso del **pipeline** que se encargará de aplicar las transformaciones que creemos convenientes al conjunto de datos para después poder aprender un modelo concreto a partir él.

Lo primero de todo será eliminar las variables , tal y como especificamos en la parte del análisis exploratorio. Eliminamos las derivadas del **radio** porque son las que dependen linealmente de este debido a la fórmula matemática. Son las siguientes: **area_mean**,**perimeter_mean**,**area_se**,**perimeter_se**,**area_worst**,**perimeter_worst**. Mientras que por el otro lado dejamos **concave_points** eliminando las que son similares a el: **concavity_mean**,**compactness_mean**,**concavity_se**,**compactness_se**,**concavity_worst** y **compactness_worst**. Con dejar una nos vale, ya que si son muy parecidad no nos aportan nada nuevo. 

Para realizar este trabajo recurrimos a uno de los transformadores del paquete de sci-kit.

In [ ]:
from sklearn.compose import make_column_transformer

column_transformer = make_column_transformer(("drop",["area_mean", "perimeter_mean", "area_se", "perimeter_se", 
                                                      "area_worst", "perimeter_worst", "concavity_mean", "compactness_mean", 
                                                      "concavity_se", "compactness_se", "concavity_worst", "compactness_worst"]),
                                             remainder="passthrough")

Otra de las transformaciones que haremos será discretizar, aunque sea un poco contradictorio en base al análisis exploratorio, ya que la única variables candidata para ser discretizada era el **radio** y tampoco estaba tan clara. De todas maneras, nuestro objetivo es ver si la hipótesis formulada es correcta o no.

In [ ]:
column_transformer_discretized = make_column_transformer(("drop",["area_mean", "perimeter_mean", "area_se", "perimeter_se", 
                                                      "area_worst", "perimeter_worst", "concavity_mean", "compactness_mean", 
                                                      "concavity_se", "compactness_se", "concavity_worst", "compactness_worst"]),
                                             ( KBinsDiscretizer(n_bins=2, strategy="uniform"), ["radius_mean","radius_worst",
                                                        "radius_se", "concave points_mean","concave points_worst","concave points_se"]),
                                             remainder="passthrough")

### Algoritmos de clasificación

Usaremos al igual que en la base de datos anterior el **Zero R** como modelo de base para evaluar los nuestros con los datos preprocesados.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

Construimos el modelo de árbol y metemos todo al pipeline que se encargará de ejecutar los pasos que le asignemos.

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

Distinguiremos dos modelos, uno con el árbol de clasificación sin discretizar tan solo con las variables eliminadas y otro con discretizado con el fin de vez cual tiene mejor rendimiento.

In [ ]:
no_columns = make_pipeline(column_transformer, tree_model)

In [ ]:
discretize_tree_model = make_pipeline(column_transformer_discretized, tree_model)

### Evaluación de los modelos

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

De nuevo y sin extrañarnos el modelo creado por el **Zero R** es un mal clasificador obteniendo un rendimiento bajo y una "línea recta" en el análisis ROC. Es nuestro punto a partida a batir.

In [ ]:
utils.evaluate(no_columns,
               X_train, X_test,
               y_train, y_test)

Hay un importante mejora en la precisión del clasificador al crear el modelo a partir de datos ya procesados. Lo cual nos muestra de nuevo la importancia de esta fase en el proceso que hemos llevado a cabo. Por otro lado, si nos fijamos en la matriz de confusión vemos que tenemos una tasa baja tanto de falsos positivos como negativo. El análisis ROC pasa con creces el punto de partida del que hemos empezado, esto se debe a lo que hemos dicho antes de las bajas tasa de falsos positivos y el incremento de los verdaderos positivos. Por tanto, hemso obtenido un muy buen clasificador.

In [ ]:
utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

Desde un principio hemos tenido dudas acerca de si llevar a cabo la discretizacion, puesto que desde el análisis exploratorio se intuía de que aplicarla sería en vano por la distribución superpuesta de las clases en todas las variables. Asi que, al fín nuestra hipotesis se cumple al ver que la precisión del clasificador disminuye respecto al modelo sin discretizar, generando más falsos positivos y negativos. Aun así la precisión es bastante alta.